# Pablo's Playhouse for Data Analysis

In [ ]:
# import libraries
import pandas as pd
import geopandas as gpd

# Define data path
dataPath_region = '../data/merged_data/merged_birthPlaceRegion_sex.csv'
dataPath_esp = '../data/merged_data/merged_birthPlace_spain_v_outside.csv'

# Load merged datasets
df_region = pd.read_csv(dataPath_region)
df_esp = pd.read_csv(dataPath_esp)










Birth Place by Region DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72898 entries, 0 to 72897
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Year_Reference      72898 non-null  int64 
 1   District            72898 non-null  object
 2   Neighborhood        72898 non-null  object
 3   Value               72898 non-null  int64 
 4   Birth_Place_Region  72898 non-null  object
 5   Sex                 72898 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.3+ MB
None

Birth Place Spain vs Outside DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317259 entries, 0 to 317258
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Year_Reference  317259 non-null  int64 
 1   District        317259 non-null  object
 2   Neighborhood    317259 non-null  object
 3   AEB             317259 non-nu

KeyError: 'Año'